In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
import torch.optim as optim

In [ ]:
train=pd.read_csv("cure_the_princess_train.csv")
test=pd.read_csv("cure_the_princess_test.csv")
validation=pd.read_csv("cure_the_princess_validation.csv")

In [ ]:
train.head(20)

In [ ]:
train.Cured.unique()

In [ ]:
train.Cured

In [ ]:
test.Cured.unique()
test.Cured

In [ ]:
validation.Cured.unique()
validation.Cured

In [ ]:
test_x = test.iloc[:,:-1]

In [ ]:
test_y = test.iloc[:,-1]

In [ ]:
train_x = train.iloc[:,:-1]

In [ ]:
train_y = train.iloc[:,-1]

In [ ]:
validation_x = validation.iloc[:,:-1]

In [ ]:
validation_y= validation.iloc[:,-1]

In [ ]:
batch_size=16
train_x = train_x.values
train_y = train_y.values

train_x = torch.tensor(train_x,dtype = torch.float32)
train_y = torch.tensor(train_y,dtype = torch.float32).reshape(-1,1)

veri_Dizisi_Train = list()
for data in range(len(train_x)):
  veri_Dizisi_Train.append([train_x[data],train_y[data]])
DL_train = torch.utils.data.DataLoader(veri_Dizisi_Train,batch_size=batch_size)




In [ ]:
test_x = test_x.values
test_y = test_y.values

test_x = torch.tensor(test_x,dtype = torch.float32)
test_y = torch.tensor(test_y,dtype = torch.float32).reshape(-1,1)

veri_Dizisi_Test = list()
for data in range(len(test_x)):
  veri_Dizisi_Test.append([test_x[data],test_y[data]])

DL_test = torch.utils.data.DataLoader(veri_Dizisi_Test,batch_size=batch_size)


In [ ]:
validation_x = validation_x.values
validation_y = validation_y.values

validation_x = torch.tensor(validation_x,dtype = torch.float32)
validation_y = torch.tensor(validation_y,dtype = torch.float32).reshape(-1,1)

veri_Dizisi_Validation = list()
for data in range(len(validation_x)):
  veri_Dizisi_Validation.append([validation_x[data],validation_y[data]])

DL_validation = torch.utils.data.DataLoader(veri_Dizisi_Validation,batch_size=batch_size)


In [ ]:
len(DL_validation)
len(DL_train)
len(DL_test)

In [ ]:
SEED = 190401088
torch.manual_seed(SEED)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out
input_size = 13
hidden_size1 = 100
hidden_size2 = 50
output_size = 1
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
batch_size = 16
num_epochs = 500
patience = 15
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
list_train_loss, list_val_loss = [], []
best_val_loss = None
patience_counter = 0
for epoch in range(num_epochs):
    train_loss = 0.0
    train_count = 0.0
    for inputs, labels in DL_train:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_count += 1.0
        train_loss += loss.item()
    val_loss = 0.0
    with torch.no_grad():
        model.eval()
        for inputs, labels in DL_validation:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    model.train()
    train_loss /= train_count
    val_loss /= len(DL_validation)
    print("Epoch", epoch, "Training loss", train_loss,"Validation Loss :",val_loss)
    list_train_loss.append(train_loss)
    list_val_loss.append(val_loss)
    val_score = val_loss
    if best_val_loss is None:
        best_val_loss = val_score # hafızada patience boyu tutmaya başla
        torch.save(model.state_dict(), "checkpoint.pt")
    elif best_val_loss < val_score: # patience counter
        patience_counter += 1
        print("Earlystopping Patience Counter:",patience_counter)
        if patience_counter == patience:
            break
    else:
        best_val_loss = val_score
        torch.save(model.state_dict(), "checkpoint.pt") # to keep the best model
        patience_counter = 0
plt.plot(list_train_loss, label="Training loss")
plt.plot(list_val_loss, label="Validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()
print('Finished Training')
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
model.load_state_dict(torch.load('checkpoint.pt'))
model.eval()
predicts =[]
real_labels = list()
with torch.no_grad():
    for inputs,label in DL_test:
        outputs = model(inputs)
        for out in outputs:
            predict = round(float(out.data))
            predicts.append(predict)
        real_labels.extend(label.tolist())
from sklearn.metrics import f1_score,accuracy_score,classification_report
print("Accuracy score of this model: {}".format(accuracy_score(real_labels,predicts)))
print(classification_report(real_labels,predicts))

In [ ]:
SEED = 190401088
torch.manual_seed(SEED)
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out) 
        out = self.fc3(out)
        out = self.sigmoid(out)
        out = out 
        return out
input_size = 13
hidden_size1 = 100
hidden_size2 = 50
output_size = 1
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
batch_size = 16
num_epochs = 500
patience = 15
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-5)
list_train_loss, list_val_loss = [], []
best_val_loss = None
patience_counter = 0
# Train the model
for epoch in range(num_epochs):

    train_loss = 0.0
    train_count = 0.0
    for inputs, labels in DL_train:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_count += 1.0
        train_loss += loss.item()
    val_loss = 0.0
    with torch.no_grad():
        model.eval()
        for inputs, labels in DL_validation:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    model.train()
    train_loss /= train_count
    val_loss /= len(DL_validation)
    print("Epoch", epoch, "Training loss", train_loss,"Validation Loss :",val_loss)
    list_train_loss.append(train_loss)
    list_val_loss.append(val_loss)
    val_score = val_loss
    if best_val_loss is None:
        best_val_loss = val_score 
        torch.save(model.state_dict(), "checkpoint.pt")
    elif best_val_loss < val_score:
        patience_counter += 1
        print("Earlystopping Patience Counter:",patience_counter)
        if patience_counter == patience:
            break
    else:
        best_val_loss = val_score
        torch.save(model.state_dict(), "checkpoint.pt")
        patience_counter = 0
print('Finished Training')
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
model.load_state_dict(torch.load('checkpoint.pt'))
model.eval()
predicts =[]
real_labels = list()
with torch.no_grad():
    for inputs,label in DL_test:
        outputs = model(inputs)
        for out in outputs:
            predict = round(float(out.data))
            predicts.append(predict)
        real_labels.extend(label.tolist())
from sklearn.metrics import f1_score,accuracy_score,classification_report
print("Accuracy score of this model: {}".format(accuracy_score(real_labels,predicts)))
print(classification_report(real_labels,predicts))